In [2]:
%config Completer.use_jedi = False
# %matplotlib widget



In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pandas as pd
import seaborn as sns
import scipy
from sklearn import metrics
import pybedtools
import coolbox
from coolbox.api import *

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects
os.chdir('/home/dcmorgan')
os.getcwd()

'/home/dcmorgan'

In [ ]:
!cut -f2,3,4 ~/data/meta/NC_004252_R0396-MWT.bedgraph

In [15]:
primary=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Primary Data')
diet=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Diet Data')
blood_stool=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='blood and stool biomarkers')
secondary=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Secondary Data')
MRI=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='MRI scores')

In [469]:
oric=pd.read_csv('data/doric10/tubic_bacteria.csv') #remove 'note' and 'nt*' from file

In [470]:
oric['oric_start']=oric['Location of replication origin'].str.split('.').str[0].str.split(',').str[0].str.split(';').str[0].tolist()
oric['oric_end']=oric['Location of replication origin'].str.split('.').str[2].str.split(',').str[0].str.split(';').str[0].tolist()
oric['gene_start']=oric['Location of replication genes'].str.split('.').str[0].str.split(',').str[0].str.split(';').str[0].tolist()
oric['gene_end']=oric['Location of replication genes'].str.split('.').str[2].str.split(',').str[0].str.split(';').str[0].tolist()

In [471]:
oric['OriC AT content']=np.round(oric['OriC AT content']*100).astype(int)
oric['orgA']=oric['Organism'].str.split(' ').str[0].tolist()
oric['orgB']=oric['Organism'].str.split(' ').str[1].tolist()
oric['Organism']=oric['orgA']+'_'+oric['orgB']


In [472]:
oric[['oric_start','oric_end','gene_start','gene_end']]=oric[['oric_start','oric_end','gene_start','gene_end']].replace('-','1')
oric=oric.fillna('1')

In [473]:
oric[['gene_start']]=oric[['gene_start']].astype(int)
oric[['gene_end']]=oric[['gene_end']].astype(int)
oric[['oric_start']]=oric[['oric_start']].astype(int)
oric[['oric_end']]=oric[['oric_end']].astype(int)

In [474]:
oric['end']=np.amax(oric[['gene_start','oric_end','gene_start','gene_end']],1)
oric['start']=np.amin(oric[['gene_start','oric_end','gene_start','gene_end']],1)

In [476]:
# oric['start']=np.amin(oric,1)#
# oric['end']=np.amax(oric,1)
# oric['index']=1
oric[['Refseq','start','end','OriC AT content','Organism']]

,Refseq,start,end,OriC AT content,Organism
0,NC_000908.1,577268,579223,72,Mycoplasma_genitalium
1,NC_000919.1,4,1640,67,Treponema_pallidum
2,NC_000964.1,410,1938,63,Bacillus_subtilis
3,NC_000964.1,409,1750,65,Bacillus_subtilis
4,NC_001263.1,1903,3268,52,Deinococcus_radiodurans
...,...,...,...,...,...
9664,NZ_CP024968,1,1556,81,Mesoplasma_coleopterae
9665,NZ_CP024968,1,800407,79,Mesoplasma_coleopterae
9666,NZ_CP024969,1,1559,82,Mesoplasma_tabanidae
9667,NZ_CP024969,1,846907,78,Mesoplasma_tabanidae


In [477]:
oric[['Refseq','start','end','OriC AT content','Organism']].to_csv('data/doric10/OriC_db.bed',sep='\t',header=True,index=False)

In [45]:
traces= glob.glob('data/meta/*bedgraph')
jac=traces[7]
ori=pd.read_csv('data/oric/OriC_db.bed',sep='\t')

In [ ]:
zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','relAb'])
tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
plt.figure(figsize=(20, 5))

plt.bar(data=zzz[(zzz['start']>3000000)&(zzz['end']<3500000)],x='start',height='relAb',label='align',color='blue')#][zzz['start']=='1665929']
plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

plt.axes([.2, .6, .6, .2])
plt.bar(data=zzz[(zzz['start']>np.int(np.round(tmp['start'].item(),-1))-100) &(zzz['end']<np.int(np.round(tmp['end'].item(),-1))+100)],x='start',height='relAb',label='align',color='blue')#][zzz['start']=='1665929']
plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')


plt.xlabel("OriC ranges from "+str(tmp['start'].item())+" to "+str(tmp['end'].item())+" for "
        +str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0])+
           "max= "+np.max(zzz['relAb'])+" at "+str(zzz[zzz['relAb']==np.max(zzz['relAb'])][['start']].index.values)+"; median= "+np.median(zzz['relAb']),fontsize=14)
 

In [ ]:
traces= glob.glob('data/meta/*bedgraph')
ori=pd.read_csv('data/doric10/OriC_db.bed',sep='\t')

for jac in traces:
    zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','relAb'])
    tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
    
    plt.figure(figsize=(20, 5))
    try:
        print('running: '+str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0]))
        ss=np.int(np.round(tmp['start'].item(),-1))
        tt=np.int(np.round(tmp['end'].item(),-1))
        plt.bar(data=zzz[(zzz['start']>ss-250000)&(zzz['end']<tt+250000)],x='start',height='relAb',label='align',color='blue')#][zzz['start']=='1665929']
        plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
        plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

        plt.axes([.2, .6, .6, .2])
        plt.bar(data=zzz[(zzz['start']>ss-100) &(zzz['end']<tt+100)],x='start',height='relAb',label='align',color='blue')#][zzz['start']=='1665929']
        plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
        plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

        plt.xlabel("OriC ranges from "+str(tmp['start'].item())+" to "+str(tmp['end'].item())+" for "
            +str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0])+"max= "+
                   np.max(zzz['relAb'])+" between "+str(np.int(zzz[zzz['relAb']==np.max(zzz['relAb'])][['start']].values))+" and "+str(np.int(zzz[zzz['relAb']==np.max(zzz['relAb'])][['end']].values))+"; median= "+np.median(zzz['relAb']),fontsize=14)
        plt.savefig('img/oric/'+str(os.path.basename(jac).split('.')[0])+'.png',dpi=300,bbox_inches = "tight")

    except TypeError:
        pass
    except ValueError:
        pass

In [25]:
ori=pd.read_csv('~/data/doric/OriC_db.bed',sep='\t')
ori

,Refseq,start,end,OriC AT content,Organism
0,NC_000908.1,577268,579223,72,Mycoplasma_genitalium
1,NC_000919.1,4,1640,67,Treponema_pallidum
2,NC_000964.1,410,1938,63,Bacillus_subtilis
3,NC_000964.1,409,1750,65,Bacillus_subtilis
4,NC_001263.1,1903,3268,52,Deinococcus_radiodurans
...,...,...,...,...,...
9664,NZ_CP024968,1,1556,81,Mesoplasma_coleopterae
9665,NZ_CP024968,1,800407,79,Mesoplasma_coleopterae
9666,NZ_CP024969,1,1559,82,Mesoplasma_tabanidae
9667,NZ_CP024969,1,846907,78,Mesoplasma_tabanidae


In [ ]:
ori[ori['Organism']=='Staphylococcus_aureus']

In [21]:
tmp

""


In [47]:
table=pd.DataFrame()
traces= glob.glob('data/meta/*bedgraph')
traces=np.sort(traces)
ori=pd.read_csv('data/doric10/OriC_db.bed',sep='\t')

for jac in traces:
    zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','relAb'])
    tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
    # print(str(tmp['Organism']))
    pat=jac.split('_')[2].split('.')[0]
    try:
        zzz['len']=zzz['end']-zzz['start']
        # zzz=zzz[zzz['len']==50]
        zzz['score']=zzz['relAb']/zzz['len']
        ss=np.int(np.round(tmp['start'].item(),-1))
        tt=np.int(np.round(tmp['end'].item(),-1))
        tmpd=zzz[(zzz['start']>ss) &(zzz['end']<tt)]
        
        # zzz
        tmp['mean']=np.mean(zzz['score'])
        tmp['ori_mean']=np.mean(tmpd['score'])

        tmp['median']=np.median(zzz['score'])
        tmp['ori_median']=np.median(tmpd['score'])
        tmp['pat']=pat
        table=pd.concat([table,tmp],axis=0)
    except ValueError:
        pass

In [48]:
table[1:50]

,Refseq,start,end,OriC AT content,Organism,mean,ori_mean,median,ori_median,pat
190,NC_002695.1,4668559,4719455,57,Escherichia_coli,0.159139,0.012113,0.01,0.0100,R0395-WHK
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,0.997062,0.842121,0.92,0.8400,R0395-WHK
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,0.736655,0.622727,0.52,0.6200,R0396-MWT
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,1.327310,1.405269,1.30,1.4800,R0397-LWS
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,6.987220,7.760000,6.52,7.7000,R0398-LKW
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,0.367141,0.195937,0.16,0.2000,R0399-SWY
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,0.398474,0.110256,0.14,0.0700,R0400-WFY
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,0.932328,0.013750,0.24,0.0075,R0401-CYW
72,NC_004307.1,1624263,1626083,54,Bifidobacterium_longum,1.107863,1.067419,1.04,1.0000,R0402-LWC
977,NC_011353.1,4771663,4822550,57,Escherichia_coli,0.160486,0.011632,0.01,0.0100,R0395-WHK


In [8]:
# pd.DataFrame(table[:,0,:])
table.to_csv('data/oriC_calc.txt',sep='\t')

In [9]:
table.groupby('pat').median()

,start,end,OriC AT content,mean,ori_mean,median,ori_median
pat,,,,,,,
R0395-WHK,2832442.0,3308811.0,57.0,0.786735,0.697153,0.42,0.500
R0396-MWT,1624263.0,3064684.0,68.0,0.718651,0.602143,0.44,0.580
R0397-LWS,1624263.0,3064684.0,68.0,2.055748,1.332500,0.58,1.220
R0398-LKW,1624263.0,3064684.0,68.0,1.619862,1.327238,0.48,0.440
R0399-SWY,1624263.0,3064684.0,68.0,1.800256,0.398170,0.48,0.200
R0400-WFY,1624263.0,3064684.0,68.0,0.485407,0.254768,0.14,0.100
R0401-CYW,865039.0,3036195.5,67.5,0.933622,0.434664,0.39,0.095
R0402-LWC,865039.0,2647588.0,61.0,1.073476,1.043594,0.60,0.990
R0403-WCM,105815.0,4343135.0,68.0,0.685638,0.693945,0.24,0.260


In [10]:
np.sum(table['mean']>table['ori_mean'])/len(table)

0.3835616438356164

In [11]:
np.sum(table['median']>table['ori_median'])/len(table)

0.1780821917808219

In [25]:
table.groupby('Organism').median()

,start,end,OriC AT content,mean,ori_mean,median,ori_median
Organism,,,,,,,
Bacteroides_fragilis,105815.0,4343135.0,68.0,0.501439,0.507116,0.12,0.12
Bifidobacterium_longum,209.0,1626083.0,54.0,0.916782,0.837984,0.69,0.73
Escherichia_coli,4703050.5,4783514.5,57.0,0.159813,0.012202,0.01,0.01
Eubacterium_rectale,3062886.0,3308811.0,73.0,1.802689,1.563768,0.55,0.82
Roseburia_intestinalis,2853127.5,2854825.5,67.5,1.399431,0.487727,0.46,0.48


In [26]:
table.groupby('Organism').mean()

,start,end,OriC AT content,mean,ori_mean,median,ori_median
Organism,,,,,,,
Bacteroides_fragilis,105815.0,4.343135e+06,68.000000,0.631363,0.641344,0.143333,0.147778
Bifidobacterium_longum,541491.0,1.485584e+06,52.666667,1.522967,1.475897,1.287500,1.401354
Escherichia_coli,4252551.5,4.777258e+06,57.000000,0.162192,0.062698,0.010000,0.010000
Eubacterium_rectale,2076320.7,3.284879e+06,73.300000,1.697426,1.300220,0.895500,0.998750
Roseburia_intestinalis,2853127.5,2.854826e+06,67.500000,1.455375,0.523001,0.490000,0.523750


In [55]:
unibact=pd.read_csv('data/328_krakenuniq_raw.csv',index_col=0)


In [57]:
unibact['spec']=(unibact.index).str.split(';').str[0:6].tolist()

In [61]:
jeff=pd.DataFrame(unibact['spec'].tolist(),columns=['tmp','tmp','tmp','tmp','tmp','species'])
jeff=jeff.fillna(method='ffill',axis=1)

In [62]:
(unibact.index).str.split(';').str[0:6].tolist()[0][5] ## transfered a bunch of these from hku to personal cpos

's__Faecalibacterium prausnitzii'

In [64]:
jeff[jeff['species'].str.contains('prevot')]#[1:50]
# jeff.ind# tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]

,tmp,tmp,tmp,tmp,tmp,species
89,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Paraprevotella,s__Paraprevotella xylaniphila
307,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Alloprevotella,s__Alloprevotella sp. E39
1115,p__Firmicutes,c__Tissierellia,o__Tissierellales,f__Peptoniphilaceae,g__Anaerococcus,s__Anaerococcus prevotii
1317,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Paraprevotella,s__Paraprevotella clara
4301,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Massiliprevotella,s__Massiliprevotella massiliensis
6068,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Alloprevotella,s__Alloprevotella sp. feline oral taxon 309
7041,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Alloprevotella,g__Alloprevotella
7971,p__Bacteroidetes,c__Bacteroidia,o__Bacteroidales,f__Prevotellaceae,g__Metaprevotella,s__Metaprevotella massiliensis


In [46]:
import duolingo
# lingo  = duolingo.Duolingo('kartik', 'my password')

SyntaxError: invalid syntax (1950362930.py, line 1)

In [ ]:
lingo  = duolingo.Duolingo('kartik', '...')
print(lingo.get_known_words('fr'))

In [ ]:
lingo  = duolingo.Duolingo('kartik', '...')
lingo.get_translations(['de', 'du'], source='de', target='fr')